To scrape data authors used Python and libraries like: Selenium and BeautifulSoup.<br>
<br>
In practice, downloading the necessary information for this analysis was not trivial. It requires a three-step approach. First, the HTML code of Polish film rankings for the years 2010-2019 was downloaded from the Filmweb using the Selenium library (this is dynamic page). Then the direct links to the movies were extracted from the above code (using BeautifulSoup). After modifying these links, they redirected directly to the cast of the given film. Eventually, the cast was scraped per film. Finally we obtained a matrix with 570 rows (movies) and their corresponding actors.

#### Dependencies loading

In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import xlsxwriter

In [19]:
list_of_the_top_films_by_year = list() #list of links to the cast of each film
list_of_actors = list() #final list of the lists of actors
html_code = list() #list for HTML codes, elements of the list refer to the year of ranking

Scraping HTML codes of rankings for each year

In [22]:
driver = webdriver.Chrome()

for i in range(2010,2020):
    driver.get(f"https://www.filmweb.pl/ranking/film/Polska/42/{i}")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    html_code.append(driver.page_source)

Extracting and modifying links to the movies from above HTML code

In [34]:
for i in html_code:
    soup = BeautifulSoup(i, 'html.parser')
    for link in soup.findAll('a', {'class': 'film__link'}):
        try:
            list_of_the_top_films_by_year.append("https://www.filmweb.pl"+link['href']+"/cast")
        except:
            pass

Creating list of the lists of actors from cast web-pages

In [35]:
for i in list_of_the_top_films_by_year:
    tmp = list()
    web_page = requests.get(i)
    soup =  BeautifulSoup(web_page.text, 'html.parser')
    for j in soup.findAll('a', {'class': 'pImg46'}):
        tmp.append(j["title"])
    list_of_actors.append(tmp)

Applying some adjustments like: characters replacing, removing films without actors.

In [54]:
list_of_actors_cleared = list()

for i in list_of_actors:
    if(len(i)!=0):
        tmp = list()
        for k in i:
            foo = k.replace(" ","_")
            foo = foo.replace("-","_")
            tmp.append(foo)
        list_of_actors_cleared.append(tmp)

Saving obtained dataset to the Excel file. After that Excel file was manually converted to CSV extension.

In [55]:
with xlsxwriter.Workbook('datasets/dataset.xlsx') as workbook:
    worksheet = workbook.add_worksheet()
    for row_num, data in enumerate(list_of_actors_cleared):
        worksheet.write_row(row_num, 0, data)